In [2]:
import gym
import gym_gridworld
from datetime import datetime

# DR TRPO related files
from grid_train_helper import *
from value import NNValueFunction
from utils import Logger
from grid_dr_policy import DRPolicyKL, DRPolicyWass

# ODRPO - KL

## No human 

In [3]:
env_name = "GridWorld-v0"
env = gym.make(env_name)
sta_num = env.observation_space.n
act_num = env.action_space.n
policy = DRPolicyKL(sta_num, act_num)
val_func = NNValueFunction(1, 5)
gamma = 0.9
lam = 1
total_eps = 50
batch_eps = 1
max_steps = 10 # max steps per episode
logger = Logger(logname=env_name + '_DR-KL_Batch=' + str(batch_eps), now=datetime.utcnow().strftime("%b-%d_%H:%M:%S"))


eps = 0
while eps < total_eps:
        trajectories = run_policy(env, policy, batch_eps, max_steps, logger)
        eps += len(trajectories)
        # add estimated values to episodes
        add_value(trajectories, val_func)  
        # calculated discounted sum of Rs
        add_disc_sum_rew(trajectories, gamma, logger)  
        # calculate advantage
        add_gae(trajectories, gamma, lam)  
        # concatenate all episodes into single NumPy arrays
        observes, actions, advantages, disc_sum_rew = build_train_set(trajectories)
        log_batch_stats(observes, actions, advantages, disc_sum_rew, eps, logger)
        disc_freqs = find_disc_freqs(trajectories, sta_num, gamma)
        all_advantages = policy.preprocess_adv(observes, actions, advantages)
        policy.update(observes, actions, all_advantages, disc_freqs, env_name, eps)
        val_func.fit(observes, disc_sum_rew, logger)
        # write logger results to file and stdout
        logger.write(display=True) 
logger.close()

Value Params -- h1: 5, h2: 5, h3: 5, lr: 0.00447
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
***** Episode 1, Mean Return = -11.0, Mean Discounted Return = -6.9 *****
ExplainedVarNew: -1.28e-05
ExplainedVarOld: -0.000133
ValFuncLoss: 8.99


***** Episode 2, Mean Return = -11.0, Mean Discounted Return = -6.9 *****
ExplainedVarNew: -0.0323
ExplainedVarOld: -0.0565
ValFuncLoss: 6.09


***** Episode 3, Mean Return = -11.0, Mean Discounted Return = -6.9 *****
ExplainedVarNew: -0.0165
ExplainedVarOld: -0.0764
ValFuncLoss: 4.93


***** Episode 4, Mean Return = -11.0, Mean Discounted Return = -6.9 *****
ExplainedVarNew: -8.18e-07
ExplainedVarOld: -2.34e-05
ValFuncLoss: 3.55


***** Episode 5, Mean Return = -11.0, Mean Discounted Return = -6.9 *****
ExplainedVarNew: -1.9e-08
ExplainedVarOld: -2.84e-07
ValFuncLoss: 3.44


***** Episode 6, Mean Return = -11.0, Mean Discounted Return = -6.9 *****
ExplainedVarNew: -4.85e-06
E

## Human in the Loop

In [4]:
env_name = "GridWorld-v0"
env = gym.make(env_name)
sta_num = env.observation_space.n
act_num = env.action_space.n
policy = DRPolicyKL(sta_num, act_num)
val_func = NNValueFunction(1, 5)
gamma = 0.9
lam = 1
total_eps = 50
batch_eps = 1
max_steps = 10 # max steps per episode
logger = Logger(logname=env_name + '_DR-KL_Batch=' + str(batch_eps), now=datetime.utcnow().strftime("%b-%d_%H:%M:%S"))


eps = 0
while eps < total_eps:
        trajectories = run_policy(env, policy, batch_eps, max_steps, logger)
        eps += len(trajectories)
        # add estimated values to episodes
        add_value(trajectories, val_func)  
        # calculated discounted sum of Rs
        add_disc_sum_rew(trajectories, gamma, logger)  
        # calculate advantage
        add_gae(trajectories, gamma, lam)  
        # concatenate all episodes into single NumPy arrays
        observes, actions, advantages, disc_sum_rew = build_train_set(trajectories)
        log_batch_stats(observes, actions, advantages, disc_sum_rew, eps, logger)
        disc_freqs = find_disc_freqs(trajectories, sta_num, gamma)
        all_advantages = policy.preprocess_adv(observes, actions, advantages)
        
        # human modifies the advantage
        # left red room
        all_advantages[0][2] += 1
        all_advantages[5][2] += 1
        all_advantages[10][1] += 1
        all_advantages[15][0] += 1
        all_advantages[20][0] += 1
        
        all_advantages[11][1] += 1
        
        # middle blue room
        all_advantages[2][2] += 1
        all_advantages[7][2] += 1
        all_advantages[12][1] += 1
        all_advantages[17][0] += 1
        all_advantages[23][0] += 1
        
        all_advantages[13][1] += 1
        
        
        policy.update(observes, actions, all_advantages, disc_freqs, env_name, eps)
        val_func.fit(observes, disc_sum_rew, logger)
        # write logger results to file and stdout
        logger.write(display=True) 
logger.close()

Value Params -- h1: 5, h2: 5, h3: 5, lr: 0.00447
***** Episode 1, Mean Return = -11.0, Mean Discounted Return = -6.9 *****
ExplainedVarNew: -0.00031
ExplainedVarOld: -0.0033
ValFuncLoss: 20.8


***** Episode 2, Mean Return = 97.0, Mean Discounted Return = 70.2 *****
ExplainedVarNew: -1.17e-07
ExplainedVarOld: -3.66e-07
ValFuncLoss: 7.1e+03


***** Episode 3, Mean Return = 97.0, Mean Discounted Return = 70.2 *****
ExplainedVarNew: -2.26e-09
ExplainedVarOld: -1.17e-07
ValFuncLoss: 6.89e+03


***** Episode 4, Mean Return = 96.0, Mean Discounted Return = 62.2 *****
ExplainedVarNew: -7.29e-11
ExplainedVarOld: -1.26e-09
ValFuncLoss: 6.08e+03


***** Episode 5, Mean Return = 95.0, Mean Discounted Return = 55.0 *****
ExplainedVarNew: -2.34e-11
ExplainedVarOld: -8.08e-11
ValFuncLoss: 5.4e+03


***** Episode 6, Mean Return = 96.0, Mean Discounted Return = 62.2 *****
ExplainedVarNew: -7.82e-12
ExplainedVarOld: -2.1e-11
ValFuncLoss: 5.8e+03


***** Episode 7, Mean Return = 95.0, Mean Discounted Re

# ODRPO - Wasserstein

## No human 

In [5]:
env_name = "GridWorld-v0"
env = gym.make(env_name)
sta_num = env.observation_space.n
act_num = env.action_space.n
policy = DRPolicyWass(sta_num, act_num)
val_func = NNValueFunction(1, 5)
gamma = 0.9
lam = 1
total_eps = 50
batch_eps = 1
max_steps = 10 # max steps per episode
logger = Logger(logname=env_name + '_DR-Wass_Batch=' + str(batch_eps), now=datetime.utcnow().strftime("%b-%d_%H:%M:%S"))


eps = 0
while eps < total_eps:
        trajectories = run_policy(env, policy, batch_eps, max_steps, logger)
        eps += len(trajectories)
        # add estimated values to episodes
        add_value(trajectories, val_func)  
        # calculated discounted sum of Rs
        add_disc_sum_rew(trajectories, gamma, logger)  
        # calculate advantage
        add_gae(trajectories, gamma, lam)  
        # concatenate all episodes into single NumPy arrays
        observes, actions, advantages, disc_sum_rew = build_train_set(trajectories)
        disc_freqs = find_disc_freqs(trajectories, sta_num, gamma)
        log_batch_stats(observes, actions, advantages, disc_sum_rew, eps, logger)
        all_advantages = policy.preprocess_adv(observes, actions, advantages)
        policy.update(observes, actions, all_advantages, disc_freqs, env_name, eps)
        val_func.fit(observes, disc_sum_rew, logger)
        # write logger results to file and stdout
        logger.write(display=True) 
logger.close()

Value Params -- h1: 5, h2: 5, h3: 5, lr: 0.00447
***** Episode 1, Mean Return = -11.0, Mean Discounted Return = -6.9 *****
ExplainedVarNew: -0.0143
ExplainedVarOld: -0.000755
ValFuncLoss: 17.7


***** Episode 2, Mean Return = -11.0, Mean Discounted Return = -6.9 *****
ExplainedVarNew: -2.12e-05
ExplainedVarOld: -2.67e-06
ValFuncLoss: 7.99


***** Episode 3, Mean Return = -11.0, Mean Discounted Return = -6.9 *****
ExplainedVarNew: -2.39e-05
ExplainedVarOld: -4.25e-05
ValFuncLoss: 4.79


***** Episode 4, Mean Return = -11.0, Mean Discounted Return = -6.9 *****
ExplainedVarNew: -6.19e-06
ExplainedVarOld: -2.39e-05
ValFuncLoss: 3.77


***** Episode 5, Mean Return = -11.0, Mean Discounted Return = -6.9 *****
ExplainedVarNew: -4.49e-06
ExplainedVarOld: -6.19e-06
ValFuncLoss: 3.5


***** Episode 6, Mean Return = -11.0, Mean Discounted Return = -6.9 *****
ExplainedVarNew: -4.22e-06
ExplainedVarOld: -4.49e-06
ValFuncLoss: 3.43


***** Episode 7, Mean Return = -11.0, Mean Discounted Return = -6.

## Human in the Loop 

In [6]:
env_name = "GridWorld-v0"
env = gym.make(env_name)
sta_num = env.observation_space.n
act_num = env.action_space.n
policy = DRPolicyWass(sta_num, act_num)
val_func = NNValueFunction(1, 5)
gamma = 0.9
lam = 1
total_eps = 50
batch_eps = 1
max_steps = 10 # max steps per episode
logger = Logger(logname=env_name + '_DR-Wass_Batch=' + str(batch_eps), now=datetime.utcnow().strftime("%b-%d_%H:%M:%S"))


eps = 0
while eps < total_eps:
        trajectories = run_policy(env, policy, batch_eps, max_steps, logger)
        eps += len(trajectories)
        # add estimated values to episodes
        add_value(trajectories, val_func)  
        # calculated discounted sum of Rs
        add_disc_sum_rew(trajectories, gamma, logger)  
        # calculate advantage
        add_gae(trajectories, gamma, lam)  
        # concatenate all episodes into single NumPy arrays
        observes, actions, advantages, disc_sum_rew = build_train_set(trajectories)
        disc_freqs = find_disc_freqs(trajectories, sta_num, gamma)
        log_batch_stats(observes, actions, advantages, disc_sum_rew, eps, logger)
        all_advantages = policy.preprocess_adv(observes, actions, advantages)
        
        # human modifies the advantage
        # left red room
        all_advantages[0][2] += 1
        all_advantages[5][2] += 1
        all_advantages[10][1] += 1
        all_advantages[15][0] += 1
        all_advantages[20][0] += 1
        
        all_advantages[11][1] += 1
        
        # middle blue room
        all_advantages[2][2] += 1
        all_advantages[7][2] += 1
        all_advantages[12][1] += 1
        all_advantages[17][0] += 1
        all_advantages[23][0] += 1
        
        all_advantages[13][1] += 1
        
        policy.update(observes, actions, all_advantages, disc_freqs, env_name, eps)
        val_func.fit(observes, disc_sum_rew, logger)
        # write logger results to file and stdout
        logger.write(display=True) 
logger.close()

Value Params -- h1: 5, h2: 5, h3: 5, lr: 0.00447
***** Episode 1, Mean Return = -11.0, Mean Discounted Return = -6.9 *****
ExplainedVarNew: -1.38e-05
ExplainedVarOld: -0.0163
ValFuncLoss: 20.1


***** Episode 2, Mean Return = 97.0, Mean Discounted Return = 70.2 *****
ExplainedVarNew: -1.37e-11
ExplainedVarOld: -1.21e-11
ValFuncLoss: 7.03e+03


***** Episode 3, Mean Return = 97.0, Mean Discounted Return = 70.2 *****
ExplainedVarNew: -4.05e-12
ExplainedVarOld: -1.37e-11
ValFuncLoss: 6.81e+03


***** Episode 4, Mean Return = 97.0, Mean Discounted Return = 70.2 *****
ExplainedVarNew: -2.01e-13
ExplainedVarOld: -4.05e-12
ValFuncLoss: 6.61e+03


***** Episode 5, Mean Return = 97.0, Mean Discounted Return = 70.2 *****
ExplainedVarNew: -3.4e-14
ExplainedVarOld: -2.01e-13
ValFuncLoss: 6.46e+03


***** Episode 6, Mean Return = 97.0, Mean Discounted Return = 70.2 *****
ExplainedVarNew: -1.75e-14
ExplainedVarOld: -3.4e-14
ValFuncLoss: 6.33e+03


***** Episode 7, Mean Return = 97.0, Mean Discounted